In [ ]:
import requests
import json
from progress.bar import Bar
import time, random


def get_payload(search_word):
    r = requests.get('https://atlas.emory.edu/api/?page=fose&route=search&keyword={}'.format(search_word),data = "{\"other\":{\"srcdb\":\"5209\"},\"criteria\":[{\"field\":\"keyword\",\"value\":\"a\"}]}")
    content = r.content.decode("utf8")
    json_c = json.loads(content)
    return json_c

def get_page(request):
    requests.adapters.DEFAULT_RETRIES = 5 # Increase the number of reconnections
    s = requests.session()
    s.keep_alive = False # Close redundant connections
    r = s.get('https://atlas.emory.edu/api/?page=fose&route=details',
                data = request)
    content = r.content.decode("utf-8")
    return json.loads(content)

payload = get_payload('cs')

courseCode = {} # get a dict with course code and crn
for course in payload['results']: # payload['results'] is a list of dict
    #course is a dict
    code = course['code']
    crn = course['crn']
    if code in courseCode:
        courseCode[code].append(crn)
    else:
        courseCode[code] = []
        courseCode[code].append(crn)
        
new_courseCode = {} # get a neat version of the dict containing course code and crn
for i in courseCode: # i is course code
    crns = courseCode[i] # crns is a list
    new_crn = ','.join(crns)
    new_courseCode[i] = new_crn

req = [] # get the request string
for i in new_courseCode:
    req.append("{\"group\":\"code:" + i + "\",\"key\":\"\",\"srcdb\":\"5209\",\"matched\":\"crn:"+new_courseCode[i]+"\"}")


total_courses = {} # containing all course data
print("Total number of requests:", len(req))
start = time.time()
for re in req:
    time.sleep(random.random()*3)
    try:
        print("|", sep=' ', end='', flush=True)
        course = get_page(re)
        newdic = {}
        newdic['courseCode'] = course['code']
        newdic['courseTitle'] = course['title']
        newdic['semester'] = course['crse_typoff_html']
        newdic['sessions'] = course['allInGroup'] # all sessions of a course
        total_courses[course['code']] = newdic
    except:
        print("Connection refused by the server..")
        print("ZZzzzz...")
        time.sleep(random.random()*5)
        print("Was a nice sleep, now let me continue...")
        continue

      
end = time.time()
timeElapse = end - start
print("timeElapse:", timeElapse)
print(newdic)  
